In [1]:
import os, sys
import numpy as np
from functools import partial

## <center>                                           Add module paths

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from MomentMatching.StateModels import GaussianState
from MomentMatching.baseMomentMatch import UnscentedTransform, TaylorTransform, MonteCarloTransform

In [3]:
x = np.array([2.0, 0.1])

### <center> Define State Distribution

In [4]:
xx_mean = np.array([1.0, 1.0], dtype=float)
xx_sigma = np.array([[1, 0], [0, 1]], dtype=float)
state_distribution = GaussianState(xx_mean, xx_sigma)


### <center>  Define a linear function 

In [5]:
A = np.array([[3, 0],
              [0, 1]], dtype=float)
B = np.array([0.0, 1.0], dtype=float)

In [6]:
def measurement_function(x, A=A, B=B):
    return np.dot(A, x) + B#[:, np.newaxis]

### <center> Define Taylor transform TT

In [7]:
TT = TaylorTransform(dimension_of_state=2)

In [8]:
TT.numerical_jacobian(f=measurement_function, x=xx_mean)

array([[ 3.,  0.],
       [ 0.,  1.]])

### <center> 

``` python 
def measurement_update(self, node, measurement):

    assert isinstance(node, TimeSeriesNodeForEP)

    measurement_cavity_distribution = node.marginal / node.measurement_factor  # q(x_t) / q_up(x_t)

    new_node = node

    new_node.marginal = self.moment_matching(self.measurement, measurement_cavity_distribution, self.R, measurement)

    new_node.measurement_factor = new_node.marginal / measurement_cavity_distribution

    return new_node


```

In [9]:
factor_mean = np.array([0.0, 0.0], dtype=float)
factor_sigma = 9999 * np.array([[1, 0], [0, 1]], dtype=float)
measurement_factor = GaussianState(factor_mean, factor_sigma)
measurement = np.array([2.0, 0.0], dtype=float)

In [10]:
measurement_cavity = state_distribution / measurement_factor

In [11]:
mz = measurement_function(measurement_cavity.mean)   # z = f(x)
C = TT.numerical_jacobian(measurement_function, measurement_cavity.mean)  # linear factor A 
sz = C @ measurement_cavity.cov @ C.T #  predictive covariance sz = var(f(x)) = A * Sigma * A^T

In [35]:
def log_gaussian(m, S, x, deriv=False):
    
    D = S.shape[0]   
    x_minus_m = x - m
    
    logexp = -0.5*np.dot(x_minus_m.T, np.linalg.solve(S, x_minus_m))
    logz = D/2.0*np.log(2.0*np.pi) + 0.5*np.log(np.linalg.det(S))
    logpdf = logexp - logz
    
#     derivatives
    
    dlogZdm = x_minus_m.T @ np.linalg.solve(S, np.eye(D, dtype=float))  # (x - m )^T * S^-1
    dlogZdS = 0.5 * (dlogZdm.T @ dlogZdm ) - np.linalg.pinv(S)/2.0
    dlogZdx = -dlogZdm
    if deriv:
        return logpdf, dlogZdm, dlogZdS, 
    else:
        return logpdf

In [36]:
logZi, dlogZidMz, dlogZidSz = log_gaussian(m=mz, x=measurement,  S=sz, deriv=True)

In [29]:
dummy = partial(log_gaussian, x=measurement, m=mz)

In [31]:
TT.numerical_jacobian(dummy, sz)

array([ 0.06175209,  1.61079171])